In [ ]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
#from MCForecastTools import MCSimulation
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
#from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import hvplot.pandas


%matplotlib inline

In [ ]:
# Load .env enviroment variables
load_dotenv()

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key=os.getenv('ALPACA_API_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [ ]:
# Format current date as ISO format
start_time=pd.Timestamp('2023-10-22', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-10-31', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["AAPL"]

# Set timeframe to "1Minute" for Alpaca API
timeframe = "15Min"


In [ ]:
def macd_order(tickers,short_window=12,long_window=26):
    df_tickers=alpaca.get_bars(
        tickers,
        timeframe,
        start=start_time,
        end=end_time
    ).df
    def calculate_macd(df_tickers = df_tickers, short_window=short_window, long_window=long_window):
        df_tickers['ShortEMA'] = df_tickers['close'].ewm(span=short_window, adjust=False).mean()
        df_tickers['LongEMA'] = df_tickers['close'].ewm(span=long_window, adjust=False).mean()
        df_tickers['MACD'] = df_tickers['ShortEMA'] - df_tickers['LongEMA']
        df_tickers['SignalLine'] = df_tickers['MACD'].ewm(span=9, adjust=False).mean()
        return df_tickers


    data_frame = calculate_macd(df_tickers, short_window, long_window)    
    position = None 
    buy_price = 0
    sell_price = 0
    signals = []
    for i in range(1, len(data_frame)):
        if data_frame['MACD'][i] > data_frame['SignalLine'][i] and data_frame['MACD'][i - 1] <= data_frame['SignalLine'][i - 1]:
            # if position is None:
                position = 'buy'
                buy_price = data_frame['close'][i]
                signals.append((data_frame.index[i], 'Buy', buy_price))
        elif data_frame['MACD'][i] < data_frame['SignalLine'][i] and data_frame['MACD'][i - 1] >= data_frame['SignalLine'][i - 1]:
            # if position == 'buy':
                position = 'sell'
                sell_price = data_frame['close'][i]
                signals.append((data_frame.index[i], 'Sell', sell_price))
    return signals




In [ ]:
#input any ticker
s= macd_order('TSLA')

In [ ]:
for signal in s:
    print(f"Signal Date: {signal[0]}, Action: {signal[1]}, Price: {signal[2]}")